In [1]:
from typing import Dict, Union
import sys
import os 
from pathlib import Path
import requests
import socket
import pandas as pd
import webbrowser
from requests_oauthlib import OAuth2Session
import gi
import tempfile
try:
    gi.require_version('NM', '1.0')
    from gi.repository import GLib, NM
except ValueError:
    print("NM not available")
    NM = None

NM not available


In [2]:
sys.path.append(str(Path().absolute().parent))

In [3]:
from eduvpn.oauth2 import one_request, get_open_port
from eduvpn.crypto import gen_code_challenge, gen_code_verifier, common_name_from_cert

# Settings

In [4]:
# client settings
DISCO_URI = 'https://disco.eduvpn.org/v2/'
ORGANISATION_URI = DISCO_URI + "organization_list.json"
SERVER_URI = DISCO_URI + "server_list.json"
client_id = "org.eduvpn.app.linux"
scope = ["config"]
code_challenge_method = "S256"

In [5]:
# our configuration
organisation = 'SURFnet bv'
server = 'Demo'
institute = 'Demo'
LANGUAGE = 'nl'
COUNTRY = "nl-NL"

# Utils

In [6]:
def extract_translation(d: Union[str, Dict[str, str]]):
    if type(d) != dict:
        return d
    for m in [COUNTRY, LANGUAGE, 'en-US', 'en']:
        try:
            return d[m]
        except KeyError:
            continue
    return list(d.values())[0]  # otherwise just return first in list

# server list

In [7]:
servers_response = requests.get(SERVER_URI)
server_list = pd.DataFrame(servers_response.json()['server_list'])
server_list['display_name'] = server_list['display_name'].apply(extract_translation)
server_list

,server_type,base_url,public_key_list,country_code,support_contact,authentication_url_template,display_name,keyword_list
0,secure_internet,https://eduvpn.rash.al/,[Xv3l24gbMX8NtTnFQbWO2fGKPwKuc6EbjQDv8qw2GVk],AL,[mailto:helpdesk@rash.al],NaN,NaN,NaN
1,secure_internet,https://gdpt-eduvpndev1.tnd.aarnet.edu.au/,[HpY5RKF0OzYcYUcogKzgt1MvC6CxBmDJoUBsyiKjioA],AU,NaN,NaN,NaN,NaN
2,secure_internet,https://eduvpn.deic.dk/,[bRTz33KIuYo_w_-AbzNtdmLDqIm11_eGiHXQniojxY4],DK,[mailto:eduvpn@deic.dk],NaN,NaN,NaN
3,secure_internet,https://eduvpn.eenet.ee/,[jGpivOdwCRoLlexYKQjulZPPP4s3d9SVBFslI6RroAo],EE,[mailto:eduvpn@eenet.ee],NaN,NaN,NaN
4,secure_internet,https://eduvpn1.funet.fi/,[H4NTnM18BJgU_B3r8OBDVblBSfozB2Zu97I_ag2whmM],FI,[mailto:eduvpn@csc.fi],https://eduvpn1.funet.fi/Shibboleth.sso/Login?...,NaN,NaN
...,...,...,...,...,...,...,...,...
60,institute_access,https://eduvpn.univ-rennes1.fr/,NaN,NaN,[https://assistance.univ-rennes1.fr/],NaN,Université de Rennes 1,NaN
61,institute_access,https://hku.eduvpn.nl/,NaN,NaN,NaN,NaN,Hogeschool voor de Kunsten Utrecht,hku
62,institute_access,https://eduvpn.vamk.fi/,NaN,NaN,[mailto:helpdesk@vamk.fi],NaN,VAMK - University of Applied Sciences,NaN
63,institute_access,https://vu.eduvpn.nl/,NaN,NaN,"[mailto:servicedesk.it@vu.nl, tel:+31205980000]",NaN,Vrije Universiteit,NaN


# institute list

In [8]:
institute_list = server_list[server_list['server_type'] == 'institute_access'].drop(['server_type'], axis=1)
institute_list

,base_url,public_key_list,country_code,support_contact,authentication_url_template,display_name,keyword_list
15,https://avans.eduvpn.nl/,NaN,NaN,"[mailto:servicepunt@avans.nl, https://servicep...",NaN,Avans Hogeschool,NaN
16,https://sunset.nuonet.fr/,NaN,NaN,[mailto:support-technique-nuo@listes.nuonet.fr],NaN,CNOUS,NaN
17,https://eduvpn-csc.funet.fi/,NaN,NaN,[mailto:eduvpn@csc.fi],NaN,CSC - IT Center for Science Ltd.,NaN
18,https://demo.eduvpn.nl/,NaN,NaN,[mailto:eduvpn@surfnet.nl],NaN,Demo,NaN
19,https://access.diak.fi/,NaN,NaN,"[mailto:tuki@diak.fi, tel:+358294696070]",NaN,DIAK,NaN
20,https://egi.eduvpn.nl/,NaN,NaN,NaN,NaN,EGI Foundation,NaN
21,https://eduvpn.ensma.fr/,NaN,NaN,[mailto:dsi@ensma.fr],NaN,Ensma,NaN
22,https://eur.eduvpn.nl/,NaN,NaN,NaN,NaN,Erasmus University Rotterdam,NaN
23,https://fontys.eduvpn.nl/,NaN,NaN,[https://fontys.nl/fontyshelpt/IT-ondersteunin...,NaN,Fontys,NaN
24,https://eduvpn.frederick.ac.cy/,NaN,NaN,"[mailto:support@frederick.ac.cy, tel:+35722394...",NaN,Frederick University,NaN


# organisation list

In [9]:
organisation_response = requests.get(ORGANISATION_URI)
organization_list = pd.DataFrame(organisation_response.json()['organization_list'])
organization_list['display_name'] = organization_list['display_name'].apply(extract_translation)
organization_list['keyword_list'] = organization_list['keyword_list'].apply(extract_translation)
organization_list

,display_name,org_id,secure_internet_home,keyword_list
0,Danish Language Council,http://idp.dsn.dk/adfs/services/trust,https://eduvpn.deic.dk/,NaN
1,Business Academy Aarhus,http://adfs.eaaa.dk/adfs/services/trust,https://eduvpn.deic.dk/,NaN
2,KMD DSPARE3,http://dans-support-04.dans-idp-dev01.northeur...,https://eduvpn.deic.dk/,NaN
3,KMD DSUP_PATCH1,https://dans-idp.kmd.dk:7080,https://eduvpn.deic.dk/,NaN
4,KMD DANSWRC2,http://dans-support-02.dans-dp-dev01.northeuro...,https://eduvpn.deic.dk/,NaN
...,...,...,...,...
732,Uganda Virus Research Institute (UVRI),https://idp.uvri.go.ug/shibboleth,https://eduvpn.renu.ac.ug/,NaN
733,Uganda Martyrs Hospital Lubaga,https://idp.lubagahospital.org/idp/shibboleth,https://eduvpn.renu.ac.ug/,NaN
734,African Center of Excellence in Bioinformatics...,https://login.ace.ac.ug/idp/shibboleth,https://eduvpn.renu.ac.ug/,NaN
735,Bishop Stuart University,https://idp.bsu.ac.ug/idp/shibboleth,https://eduvpn.renu.ac.ug/,NaN


# Secure internet

In [10]:
secure_internet_list = server_list[server_list['server_type'] == 'secure_internet'].drop(['server_type'], axis=1)
secure_internet_list

,base_url,public_key_list,country_code,support_contact,authentication_url_template,display_name,keyword_list
0,https://eduvpn.rash.al/,[Xv3l24gbMX8NtTnFQbWO2fGKPwKuc6EbjQDv8qw2GVk],AL,[mailto:helpdesk@rash.al],NaN,NaN,NaN
1,https://gdpt-eduvpndev1.tnd.aarnet.edu.au/,[HpY5RKF0OzYcYUcogKzgt1MvC6CxBmDJoUBsyiKjioA],AU,NaN,NaN,NaN,NaN
2,https://eduvpn.deic.dk/,[bRTz33KIuYo_w_-AbzNtdmLDqIm11_eGiHXQniojxY4],DK,[mailto:eduvpn@deic.dk],NaN,NaN,NaN
3,https://eduvpn.eenet.ee/,[jGpivOdwCRoLlexYKQjulZPPP4s3d9SVBFslI6RroAo],EE,[mailto:eduvpn@eenet.ee],NaN,NaN,NaN
4,https://eduvpn1.funet.fi/,[H4NTnM18BJgU_B3r8OBDVblBSfozB2Zu97I_ag2whmM],FI,[mailto:eduvpn@csc.fi],https://eduvpn1.funet.fi/Shibboleth.sso/Login?...,NaN,NaN
5,https://eduvpn-poc.renater.fr/,[ePVNzE15h0yS6Xf3s8nJWmc8V6FeFziA3TZr0uOacFg],FR,[https://assistance.renater.fr/],https://eduvpn-poc.renater.fr/Shibboleth.sso/L...,NaN,NaN
6,https://eduvpn1.eduvpn.de/,[QjJHMit3vhHwLKi-fu2dXXSxMxnkskFVS3hMwyCnWQs],DE,"[mailto:eduvpn@dfn.de, tel:+49308842999120]",https://eduvpn1.eduvpn.de/saml/login?ReturnTo=...,NaN,NaN
7,https://eduvpn.myren.net.my/,[2WsUQpiUofRZ9IWZ8qQaOeQG4-RMy981QguOEMg4-e4],MY,[mailto:helpdesk@myren.net.my],https://eduvpn.myren.net.my/Shibboleth.sso/Log...,NaN,NaN
8,https://eduvpn.marwan.ma/,[aX-El_yRPdcUDF5S2smQ-9U7BzB35_1RtFYSjbHfEz8],MA,"[mailto:eduvpn-support@marwan.ma, tel:+2127000...",https://eduvpn.marwan.ma/saml/login?ReturnTo=@...,NaN,NaN
9,https://guest.eduvpn.no/,[qOLCcqXWZm9nmjsrwiJQxxWD606vDEJ2MIcc85oJmnE],NO,[mailto:kontakt@uninett.no],NaN,NaN,NaN


# make selection

In [11]:
institute_info = institute_list[institute_list['display_name'] == institute]
institute_info

,base_url,public_key_list,country_code,support_contact,authentication_url_template,display_name,keyword_list
18,https://demo.eduvpn.nl/,NaN,NaN,[mailto:eduvpn@surfnet.nl],NaN,Demo,NaN


## or in case you select an organisation

In [12]:
organisation_info = organization_list[organization_list['display_name'] == organisation]
organisation_info

,display_name,org_id,secure_internet_home,keyword_list
711,SURFnet bv,https://idp.surfnet.nl,https://nl.eduvpn.org/,SURFnet bv SURF konijn powered by


In [13]:
info_base_url = institute_info['base_url'].iloc[0]

In [14]:
info_url = info_base_url + '.well-known/vpn-user-portal'
info = requests.get(info_url).json()['api']['http://eduvpn.org/api#2']

In [15]:
api_base_uri = info['api_base_uri']
token_endpoint = info['token_endpoint']
authorization_endpoint = info['authorization_endpoint']

In [16]:
port = get_open_port()
redirect_uri = f'http://127.0.0.1:{port}/callback'

In [17]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, auto_refresh_url=token_endpoint, scope=scope)

In [18]:
code_verifier = gen_code_verifier()
code_challenge = gen_code_challenge(code_verifier)
authorization_url, state = oauth.authorization_url(url=authorization_endpoint,
                                                   code_challenge_method=code_challenge_method,
                                                   code_challenge=code_challenge)

In [19]:
webbrowser.open(authorization_url)
response = one_request(port, lets_connect=False)

127.0.0.1 - - [06/Nov/2020 14:36:14] "GET /callback?code=eyJhbGciOiJFZERTQSIsInR5cCI6IkpXVCIsImtpZCI6IldMQTA5RWVfWHdoU0ZYalBOZ2ExT3lXRW40a1pUZ2xNR3ppR1RtRnM5MEEifQ.eyJ2Ijo1LCJ0eXBlIjoiYXV0aG9yaXphdGlvbl9jb2RlIiwiYXV0aF9rZXkiOiJjOThlMzRhZWIyOTkyZTU0N2NhNDJjNDRhMzYzNWRjZSIsInVzZXJfaWQiOiJodHRwczpcL1wvc2EtZ3cuc3VyZmNvbmV4dC5ubFwvYXV0aGVudGljYXRpb25cL21ldGFkYXRhIWh0dHBzOlwvXC9kZW1vLmVkdXZwbi5ubFwvc2FtbCFlMTEzN2VmNzQyYWRmODMwOGFhYjdlMGE2YjY0M2M4ZTIyY2JmM2ZiIiwiY2xpZW50X2lkIjoib3JnLmVkdXZwbi5hcHAubGludXgiLCJzY29wZSI6ImNvbmZpZyIsInJlZGlyZWN0X3VyaSI6Imh0dHA6XC9cLzEyNy4wLjAuMTo1Mzk4M1wvY2FsbGJhY2siLCJjb2RlX2NoYWxsZW5nZSI6IlNDWXNGTTM0S2huQ2xqb1RPT05xVmdUcXhPT0JEX3B2R0wzcHVHUTJyNVUiLCJleHBpcmVzX2F0IjoiMjAyMC0xMS0wNlQxMjo0MToxMyswMDowMCJ9.FWoaLGh1QEneSd8uZF04XEHMsbNozBEJBNspCeFw0W9xHDLeyjp9sXC2SjaBBlSlx14GNBwXtSBDpw_4RrW4Cw&state=b73GF3UhmoWZorTgwT2O2AuzuzCHnN HTTP/1.1" 200 -


In [20]:
code = response['code'][0]
assert(state == response['state'][0])

In [21]:
token = oauth.fetch_token(token_url=token_endpoint,
                          code=code,
                          code_verifier=code_verifier,
                          client_id=oauth.client_id,
                          include_client_id=True,
                          )

# profile list

In [22]:
profile_list_response = oauth.get(api_base_uri + '/profile_list')

In [23]:
profile_list_response.json()['profile_list']['data']

[{'profile_id': 'internet',
  'display_name': 'internet',
  'two_factor': False,
  'default_gateway': True},
 {'profile_id': 'routes',
  'display_name': 'No rfc1918 routes',
  'two_factor': False,
  'default_gateway': False}]

In [24]:
response = oauth.post(api_base_uri + '/create_keypair')
keypair = response.json()['create_keypair']['data']
private_key = keypair['private_key']
certificate = keypair['certificate']
common_name = common_name_from_cert(certificate.encode('ascii'))

# profile config

In [25]:
profile_id = 'internet'
response = oauth.get(api_base_uri + f'/profile_config?profile_id={profile_id}')
config = response.text

# check_certificate

In [26]:
response = oauth.get(api_base_uri + f'/check_certificate?common_name={common_name}')
assert(response.json()['check_certificate']['data']['is_valid'])

# system_messages

In [27]:
response = oauth.get(api_base_uri + '/system_messages')

In [28]:
response.json()['system_messages']['data']

[{'type': 'notification',
  'date_time': '2019-05-16T07:59:48Z',
  'message': 'Welcome to the eduVPN demo server by SURFnet. If you have any questions, please let us know by mailing us on eduvpn@surfnet.nl.'}]

# write networkmanager config

In [29]:
tmp = tempfile.NamedTemporaryFile(mode='w+t')
tmp.writelines(config)
tmp.seek(0)
filename = tmp.name

In [30]:
connection = None
for vpn_info in NM.VpnPluginInfo.list_load():
    print("TRY:  plugin %s" % (vpn_info.get_filename()))
    try:
        vpn_plugin = vpn_info.load_editor_plugin()
    except Exception as e:
        print("SKIP: cannot load plugin: %s" % (e))
        continue
    try:
        connection = vpn_plugin.import_(filename)
    except Exception as e:
        print("SKIP: failure to import %s" % (e))
        continue
    break

if connection is None:
    print("None of the VPN plugins was able to import \"%s\"" % (filename))
    sys.exit(1)

AttributeError: 'NoneType' object has no attribute 'VpnPluginInfo'

In [ ]:
connection.normalize()

print("connection imported from \"%s\" using plugin \"%s\" (\"%s\", %s)" % (filename, vpn_info.get_filename(), connection.get_id(), connection.get_uuid()))

client = NM.Client.new(None)

main_loop = GLib.MainLoop()

def added_cb(client, result, data):
    try:
        client.add_connection_finish(result)
        print("The connection profile has been successfully added to NetworkManager.")
    except Exception as e:
        print("ERROR: failed to add connection: %s\n" % e)
    main_loop.quit()

client.add_connection_async(connection, True, None, added_cb, None)

main_loop.run()